In [ ]:
import scipp as sc
import scippneutron as scn
import numpy as np
import h5py as h5
from shutil import copyfile
import sys
from ipyfilechooser import FileChooser

In [ ]:
def fix_nexus_scipp(infile, outfile):
    """
    Currently there are some tweaks to make file working in scipp. This function won't be necessary in the future
    """

    copyfile(infile, outfile)
    with h5.File(outfile, 'r+') as f:
         
        group = f['entry/instrument']
        for monitor_name in filter(lambda k: k.startswith('monitor'), group):
            monitor_group = group[monitor_name]
            monitor_event_group = monitor_group[f'{monitor_name}_events']
            for key in list(monitor_event_group):
                monitor_group[key] = monitor_event_group.pop(key)
            del monitor_group[f'{monitor_name}_events']


In [ ]:
def fix_nexus_mantid(infile, outfile, didx):
    """
    Additional tweaks for loading into Mantid
    """

    copyfile(infile, outfile)
    with h5.File(outfile, 'r+') as f:
        
        f['entry/instrument/name'] = 'LARMOR'

        group_entry = f['entry']
        nx_class = group_entry.create_group('sample')
        nx_class.attrs["NX_class"] = 'NXsample'
       
        f['entry/larmor_detector_events'] = f['entry/instrument/larmor_detector/larmor_detector_events']
        
        #TODO: Adding detector_id. Not sure if this is 100% correct
        f['entry']['instrument']['monitor_1']['detector_id'] = np.array(didx+1)
        f['entry']['instrument']['monitor_2']['detector_id'] = np.array(didx+2)
        
        group = f['entry/instrument']
        for monitor_name in filter(lambda k: k.startswith('monitor'), group):
            monitor_group = group[monitor_name]
            monitor_event_group = monitor_group[f'{monitor_name}_events']
            for key in list(monitor_event_group):
                monitor_group[key] = monitor_event_group.pop(key)
            del monitor_group[f'{monitor_name}_events']


In [ ]:
# Create and display a FileChooser widget
fc = FileChooser('/Users/wojciechpotrzebowski/SCIPP_SPACE/loki_detector_test/2022-06-24_calibrated_nexus_files/')
display(fc)

### Load and show data

In [ ]:
input_file = fc.selected
fixed_file = f'{input_file[:-4]}_fixed.nxs'
fixed_mantid_file = f'{input_file[:-4]}_mantid.nxs'

In [ ]:
fix_nexus_scipp(input_file, fixed_file)

In [ ]:
data = scn.load_nexus(data_file=fixed_file)

In [ ]:
data

## Tweaking file so it works in Mantid

In [ ]:
fix_nexus_mantid(input_file, fixed_mantid_file, len(data.coords['position']))

### Instrument view

In [ ]:
scn.instrument_view(data, pixel_size=0.01, norm='log')

### Histograming data

In [ ]:
start_tof = data.coords['tof'][0].values
end_tof = data.coords['tof'][-1].values
nbins = 100

In [ ]:
tof_edges = sc.linspace(dim='tof', start=start_tof, stop=end_tof, num=nbins, unit='ns')
histogrammed = sc.histogram(data, bins=tof_edges)

In [ ]:
histogrammed.sum('detector_id').plot()

In [ ]:
scn.instrument_view(histogrammed, pixel_size=0.01, norm='log')

## Sorting out monitors

In [ ]:
monitors = {
    'sample': {'incident': data.attrs["monitor_1"].value,
               'transmission': data.attrs["monitor_2"].value}
}

In [ ]:
monitors['sample']['incident'].coords['position'] = sc.vector(value = np.array([0, 0, 0.0]), unit = sc.units.m)
monitors['sample']['incident'].coords['source_position'] = sc.vector(value = np.array([0, 0, -25.3]), unit = sc.units.m)
monitors['sample']['transmission'].coords['position'] = sc.vector(value = np.array([0, 0, 0.0]), unit = sc.units.m)
monitors['sample']['transmission'].coords['source_position'] = sc.vector(value = np.array([0, 0, -25.3]), unit = sc.units.m)

In [ ]:
tof_edges = sc.linspace(dim='tof', start=start_tof, stop=end_tof, num=nbins, unit='ns')
histogrammed_monitor1 = sc.histogram(monitors['sample']['incident'], bins=tof_edges)
histogrammed_monitor2 = sc.histogram(monitors['sample']['transmission'], bins=tof_edges)

In [ ]:
sc.plot({'monitor1':histogrammed_monitor1, 'monitor2':histogrammed_monitor2})